In [1]:
#Setup Dependencies 
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import requests
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\emily\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [3]:
# Begin scraping
url = 'https://mars.nasa.gov/news/'

# Reach out to site
browser.visit(url)

html = browser.html

news_data = BeautifulSoup(html, 'html.parser')

In [4]:
#Find title
news_title = news_data.find_all('div', class_ = 'content_title')[1].text

news_paragraph = news_data.find_all('div', class_ = 'article_teaser_body')[0].text


In [5]:
print(news_title)
print(news_paragraph)

NASA Ingenuity Mars Helicopter Prepares for First Flight
Now uncocooned from its protective carbon-fiber shield, the helicopter is being readied for its next steps.  


In [6]:
#Head to the other site to get the space images 
nasa_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/'

next_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'

browser.visit(next_url)

html = browser.html

image = BeautifulSoup(html, 'html.parser')

In [7]:
image_url = image.find_all('img')[2]["src"]

print(image_url)

featured_image_url = nasa_url + image_url

print(featured_image_url)

image/mars/Icaria Fossae7.jpg
https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/mars/Icaria Fossae7.jpg


In [11]:
#Visit Mars Facts page 
mars_url = 'https://space-facts.com/mars/'

facts = pd.read_html(mars_url)

facts

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [12]:
#convert data to HTML table string -- first put it into a table then put to html 
facts_df = facts[2]

facts_df.columns = ["Description", "Value"]

html_table = facts_df.to_html()

In [13]:
print(html_table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Description</th>
      <th>Value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
    

In [21]:
#Find information on Mars Atmosphere 
a_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
b_url = 'https://astrogeology.usgs.gov/'
browser.visit(a_url)

html = browser.html

atmosphere = BeautifulSoup(html, 'html.parser')


In [16]:
all_atmospheres = atmosphere.find('div', class_ = 'result-list')

atmosphere_item = all_atmospheres.find_all('div', class_= 'item')


In [25]:
atmosphere_urls = []

for i in atmosphere_item:
    atmosphere = i.find('div', class_= 'description')
    title = atmosphere.h3.text
    print(title)

    atmosphere_link = atmosphere.a["href"]
    browser.visit(b_url + atmosphere_link)

    image_html = browser.html
    image_soup = BeautifulSoup(image_html, 'html.parser')

    image_link = image_soup.find('div', class_ = 'downloads')

    image_url = image_link.find('li').a['href']
    print(image_url)

    image_dict = {}
    image_dict['title'] = title
    image_dict['imgage_url'] = image_url

    atmosphere_urls.append(image_dict)

print(atmosphere_urls)

Cerberus Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
Schiaparelli Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
Syrtis Major Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
Valles Marineris Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg
[{'title': 'Cerberus Hemisphere Enhanced', 'imgage_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'imgage_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'imgage_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'ti